In [1]:
from ultralytics import YOLO
import os

model = YOLO('best.pt')

os.makedirs('runs',exist_ok=True)
os.makedirs('export',exist_ok=True)

In [2]:
result = model.val(data='../Lab2/data/face-mask-detection-yolo/data.yaml', batch=1, project='runs', save_json=True)

Ultralytics 8.3.27  Python-3.10.11 torch-2.0.0+cu117 CPU (AMD Ryzen 5 3500U with Radeon Vega Mobile Gfx)


Model summary (fused): 168 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\Users\artem\Some stuff\MLOps\Lab2\data\face-mask-detection-yolo\labels\val.cache... 171 images, 0 backgrounds, 0 corrupt: 100%|██████████| 171/171 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 171/171 [00:17<00:00,  9.69it/s]


                   all        171        729      0.884      0.683      0.764      0.512
 mask_weared_incorrect         23         24      0.865      0.625      0.694      0.478
             with_mask        151        604      0.941      0.821      0.908      0.616
          without_mask         49        101      0.845      0.604      0.689      0.441
Speed: 1.3ms preprocess, 92.4ms inference, 0.0ms loss, 1.0ms postprocess per image
Saving runs\val27\predictions.json...
Results saved to runs\val27


In [3]:
model.export(format='onnx', batch=1)

from onnxsim import simplify
import onnx

# Load ONNX model
model_path = "best.onnx"  
onnx_model = onnx.load(model_path)

# Simplify the model
simplified_model, check = simplify(onnx_model)

if check:
    onnx.save(simplified_model, "best_op.onnx")
    print("ONNX model simplified and saved.")
else:
    print("Simplification failed.")


Ultralytics 8.3.27  Python-3.10.11 torch-2.0.0+cu117 CPU (AMD Ryzen 5 3500U with Radeon Vega Mobile Gfx)

PyTorch: starting from 'best.pt' with input shape (1, 3, 416, 416) BCHW and output shape(s) (1, 7, 3549) (5.9 MB)

ONNX: starting export with onnx 1.17.0 opset 17...
============= Diagnostic Run torch.onnx.export version 2.0.0+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

ONNX: slimming with onnxslim 0.1.39...
ONNX: export success  2.3s, saved as 'best.onnx' (11.6 MB)

Export complete (2.6s)
Results saved to C:\Users\artem\Some stuff\MLOps\Lab3
Predict:         yolo predict task=detect model=best.onnx imgsz=416  
Validate:        yolo val task=detect model=best.onnx imgsz=416 data=data/face-mask-detection-yolo/data.yaml  
Visualize:       https://netron.app
ONNX model simplified and saved.


In [4]:
onnx_model = YOLO('best_op.onnx')

results = onnx_model.val(data='../Lab2/data/face-mask-detection-yolo/data.yaml', imgsz=416, batch=1, project='runs', save_json=True)

WARNING  Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Ultralytics 8.3.27  Python-3.10.11 torch-2.0.0+cu117 CPU (AMD Ryzen 5 3500U with Radeon Vega Mobile Gfx)
Loading best_op.onnx for ONNX Runtime inference...
Preferring ONNX Runtime AzureExecutionProvider
Setting batch=1 input of shape (1, 3, 416, 416)


val: Scanning C:\Users\artem\Some stuff\MLOps\Lab2\data\face-mask-detection-yolo\labels\val.cache... 171 images, 0 backgrounds, 0 corrupt: 100%|██████████| 171/171 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 171/171 [00:10<00:00, 15.77it/s]


                   all        171        729      0.935      0.651      0.751      0.501
 mask_weared_incorrect         23         24      0.932      0.574      0.661      0.454
             with_mask        151        604      0.946      0.806      0.904      0.613
          without_mask         49        101      0.928      0.574      0.689      0.438
Speed: 1.8ms preprocess, 52.6ms inference, 0.0ms loss, 1.2ms postprocess per image
Saving runs\val28\predictions.json...
Results saved to runs\val28


In [6]:

results = onnx_model.predict("test6.jpg", imgsz=416)
#print(res)
res_m = model.predict("test6.jpg", imgsz=416)
#print(res_m)


image 1/1 c:\Users\artem\Some stuff\MLOps\Lab3\test6.jpg: 416x416 1 mask_weared_incorrect, 7 with_masks, 57.3ms
Speed: 4.0ms preprocess, 57.3ms inference, 5.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 c:\Users\artem\Some stuff\MLOps\Lab3\test6.jpg: 288x416 1 mask_weared_incorrect, 7 with_masks, 96.8ms
Speed: 3.0ms preprocess, 96.8ms inference, 1.0ms postprocess per image at shape (1, 3, 288, 416)


In [7]:
import cv2

image_path = "test6.jpg"
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

for result in results:
    boxes = result.boxes.xyxy  # Bounding boxes (x1, y1, x2, y2)
    confs = result.boxes.conf  # Confidence scores
    labels = result.boxes.cls  # Class labels

    # Draw bounding boxes
    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = map(int, box.tolist())  # Convert box coordinates to integers
        conf = confs[i]
        cls = int(labels[i])  # Class index
        label = f"{result.names[cls]} {conf:.2f}"  # Class name and confidence

        # Draw the rectangle
        cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)

        # Add the label
        cv2.putText(
            image,
            label,
            (x1, y1 - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (255, 0, 0),
            2,
        )

# Show the image with bounding boxes
cv2.imshow("Detections", cv2.cvtColor(image, cv2.COLOR_RGB2BGR))  # Convert back to BGR for OpenCV display
cv2.waitKey(0)
cv2.destroyAllWindows()